<a href="https://colab.research.google.com/github/Jegovila/cursoVR/blob/main/Practica5%3A%20Detecci%C3%B3n%20de%20esquinas/python/Pr%C3%A1ctica5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc "https://raw.githubusercontent.com/Jegovila/cursoVR/main/Practica5%3A%20Detecci%C3%B3n%20de%20esquinas%20/data/edificio.jpg"

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

Filtros

In [3]:
# ----- BOX -----
n = 17
h = np.ones((n, n), np.float32)/(n*n)
# ----Sobel------
hx = np.matrix('-1 0 1;-2 0 2;-1 0 1')
hy = np.matrix('-1 -2 -1;0 0 0;1 2 1')
# ----- Gauss ----
hg = np.matrix('1 4 6 4 1;4 16 24 16 4;6 24 36 24 6;4 16 24 16 4;1 4 6 4 1')
hg = hg * (1/256)

Leer imagen

In [125]:
I = cv2.imread("edificio.jpg")
Ig = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
Ig = Ig.astype(np.float64) # convertir a float, de lo contrario los calculos se redondean
M, N = Ig.shape

Parámetros del detector

In [163]:
alpha = 0.0014
th = 50
vecindad = 50

In [158]:
S = np.zeros((M, N), dtype=np.float64)

# Suavizado
Ig = cv2.filter2D(Ig, -1, h)

# Derivadas
Ix = cv2.filter2D(Ig, -1, hx)
Iy = cv2.filter2D(Ig, -1, hy)
# Ixx = cv2.filter2D(Ix, -1, hx)

HE11 = np.multiply(Ix,Ix)
HE22 = np.multiply(Iy,Iy)
HE12 = np.multiply(Ix,Iy)

A = cv2.filter2D(HE11, -1, hg)
B = cv2.filter2D(HE22, -1, hg)
C = cv2.filter2D(HE12, -1, hg)

In [159]:
V = (A * B) - (C**2) - alpha * ((A * B) * (A * B))
U = V > th

Supresión no máxima

In [160]:
for r in range(M):
    for c in range(N):
        if U[r, c]:
            I1 = np.array([r - vecindad, 0])
            I2 = np.array([r + vecindad, M])
            I3 = np.array([c - vecindad, 0])
            I4 = np.array([c + vecindad, N])

            datxi = np.max(I1)
            datxs = np.min(I2)
            datyi = np.max(I3)
            datys = np.min(I4)

            MaxB = np.max(V[datxi:datxs, datyi:datys])

            if V[r, c] == MaxB:
                S[r, c] = 1

Dibujar esquinas

In [161]:
for r in range(M):
    for c in range(N):
        if S[r, c]:
            # cv2.drawMarker(img, (x, y), color, markerType, markerSize, thickness)
            cv2.drawMarker(I, (c, r), (0, 255, 0), cv2.MARKER_CROSS, 15, 2)

In [ ]:
cv2_imshow(I)